In [7]:
import tensorflow as tf
import keras as kr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from functools import reduce

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, Input, MaxPooling2D, AveragePooling2D

%matplotlib inline

In [8]:
NUMBER_SIZE = 15
THRESHOLD_COUNT = 5

BATCH_NUMBER_COUNT_50 = 50
SINGLE_SAMPLE_SIZE_50 = BATCH_NUMBER_COUNT_50 * NUMBER_SIZE

CLASSES = { "none": [1,0,0,0], "sparse": [0,1,0,0], "head-heavy": [0,0,1,0], "tail-heavy": [0,0,0,1] }

In [9]:
def create_single_sample_from_array(numbers):
  # Treat the list of phone numbers as a single huge list digits
  # pretty much like greyscale images, etc.
  # We then specify the number boundaries in the conv net
  digit_str = ''.join([str(i) for i in numbers])
  # Now create an image-like array
  return list(digit_str)

def create_single_sample_from_dataframe(dataframe_row):
  return create_single_sample_from_array(np.array(dataframe_row))

def combine_dfs(dfs, outcome_values=None, shuffle=True):# Combine data with sequence with outcome
  df_list = []
  for idx, df in enumerate(dfs):
    data_arr = []
    for index, series in df.iterrows():
      series_value = [v for _, v in series.iteritems()]
      combined_series = create_single_sample_from_array(series_value)
      data_arr.append(combined_series)
    
    df_arr_row_count = len(data_arr)
            
    outcome_value = outcome_values[idx] if outcome_values else idx
    data_outcome_tuples = zip(data_arr, [outcome_value] * df_arr_row_count)

    # DEBUG
    print("outcome_value:%s, outcome_values:%s" % (outcome_value, outcome_values))

    df_list += data_outcome_tuples   
    
  # Merge data with sequence and no sequence
  dataframe = pd.DataFrame(df_list, columns=["sample","outcome"])
  # Shuffle (or rather randomly select samples) but 1.0 means all
  df_random = dataframe.sample(frac=1) if shuffle else dataframe
  return df_random

def extract_sample_and_outcome(df, sample_col_name='sample', outcome_col_name='outcome'):
  X = [i for i in df[sample_col_name]]
  # Y = [int(i) for i in df[outcome_col_name]]
  Y = [i for i in df[outcome_col_name]]
  return (X, Y)

def load_data(filename, expected_shape_tuple):
  data = pd.read_csv(filename, header=None)
  print("data file:%s, data.shape (should be %s): %s" % (filename, expected_shape_tuple, data.shape))
  if data.shape != expected_shape_tuple:
    raise ValueError("data.shape:%s does not match excpected shape:%s" % (data.shape, expected_shape_tuple))
  return data
    
def prepare_data(filenames, expected_shape_tuples, outcome_values=None, shuffle=True):
  dfs = []
  for idx, filename in enumerate(filenames):
    dfs.append(load_data(filename, expected_shape_tuples[idx]))

  df_all = combine_dfs(dfs, outcome_values, shuffle)
  if (df_all.shape[0] != reduce((lambda m, i: m + i.shape[0]), dfs,0)):
    raise ValueError("There is a problem with combine_seq_no_seq. df_all.shape:%s does not match the sum of df_seq:%s and df_no_seq:%s" % (df_all.shape, df_seq.shape, df_no_seq.shape))
  return df_all

def prepare_train_data(dataframe, single_sample_size):
  df_row_count = dataframe.shape[0]
  (X, Y) = extract_sample_and_outcome(dataframe)
  X_train = np.array(X).reshape(df_row_count, 1, single_sample_size, 1)
  Y_train = np.array(Y).reshape(df_row_count, len(CLASSES.keys()))

  print("type(X):%s, len(X):%d, len(X[0]):%d" % (type(X), len(X), len(X[0])))
  print("df_row_count: %d" % df_row_count)
  print("single_sample_size_bit:%d" % single_sample_size)
  print("X_train.shape:%s, Y_train.shape:%s" % (X_train.shape, Y_train.shape))
  print("\n")

  print("type(Y[0]):%s, Y[0]:%s" % (type(Y[0]), Y[0]))

  return (X_train, Y_train)

In [4]:
# Load data from files into dataframe

df_random_with_mid_ooo_2000_2000_2000_2000_sample_number_50 = prepare_data(['data_no_sequence_2000_sample_number_50.csv', 'data_sequence_sparse_ooo_mid_combo_2000_sample_number_50.csv', 'data_sequence_head_heavy_ooo_mid_combo_2000_sample_number_50.csv', 'data_sequence_tail_heavy_ooo_mid_combo_2000_sample_number_50.csv'], [(2000,50), (2000,50), (2000,50), (2000,50)], [CLASSES["none"], CLASSES["sparse"], CLASSES["head-heavy"], CLASSES["tail-heavy"]])

df_random_with_mid_ooo_10000_10000_10000_10000_sample_number_50 = prepare_data(['data_no_sequence_10000_sample_number_50.csv', 'data_sequence_sparse_ooo_mid_combo_10000_sample_number_50.csv', 'data_sequence_head_heavy_ooo_mid_combo_10000_sample_number_50.csv', 'data_sequence_tail_heavy_ooo_mid_combo_10000_sample_number_50.csv'], [(10000,50), (10000,50), (10000,50), (10000,50)], [CLASSES["none"], CLASSES["sparse"], CLASSES["head-heavy"], CLASSES["tail-heavy"]])

data file:data_no_sequence_2000_sample_number_50.csv, data.shape (should be (2000, 50)): (2000, 50)
data file:data_sequence_sparse_ooo_mid_combo_2000_sample_number_50.csv, data.shape (should be (2000, 50)): (2000, 50)
data file:data_sequence_head_heavy_ooo_mid_combo_2000_sample_number_50.csv, data.shape (should be (2000, 50)): (2000, 50)
data file:data_sequence_tail_heavy_ooo_mid_combo_2000_sample_number_50.csv, data.shape (should be (2000, 50)): (2000, 50)
outcome_value:[1, 0, 0, 0], outcome_values:[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
outcome_value:[0, 1, 0, 0], outcome_values:[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
outcome_value:[0, 0, 1, 0], outcome_values:[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
outcome_value:[0, 0, 0, 1], outcome_values:[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
data file:data_no_sequence_10000_sample_number_50.csv, data.shape (should be (10000, 50)): (10000, 50)
data file:data_sequence_sparse_ooo_

In [5]:
# SANITY CHECK #1

# There should be 'sample' and 'outcome' columns
# 'sample' contains all phone numbers in period broken down into individual digits
# and combined
# 'outcome' is 0 or 1 for binary
# Also there should be mixed 0 and 1 since the samples have been randomize

df_random_with_mid_ooo_2000_2000_2000_2000_sample_number_50.head(20)

,sample,outcome
4812,"[8, 0, 3, 6, 0, 6, 3, 7, 1, 9, 9, 7, 9, 0, 0, ...","[0, 0, 1, 0]"
2635,"[6, 3, 5, 3, 3, 2, 4, 6, 3, 0, 1, 8, 4, 4, 6, ...","[0, 1, 0, 0]"
5445,"[9, 2, 8, 1, 8, 8, 8, 7, 6, 2, 5, 3, 2, 0, 8, ...","[0, 0, 1, 0]"
5948,"[2, 3, 8, 2, 7, 1, 7, 7, 1, 2, 8, 1, 0, 6, 4, ...","[0, 0, 1, 0]"
6061,"[2, 8, 9, 8, 2, 8, 7, 7, 4, 2, 9, 7, 3, 3, 2, ...","[0, 0, 0, 1]"
3028,"[6, 1, 5, 2, 7, 2, 0, 9, 7, 8, 0, 3, 2, 3, 2, ...","[0, 1, 0, 0]"
6503,"[2, 6, 0, 5, 8, 3, 0, 0, 6, 8, 9, 5, 8, 5, 2, ...","[0, 0, 0, 1]"
1354,"[1, 1, 7, 8, 2, 9, 1, 0, 9, 7, 3, 9, 8, 3, 2, ...","[1, 0, 0, 0]"
3823,"[1, 8, 7, 0, 7, 7, 5, 4, 3, 3, 6, 8, 5, 7, 5, ...","[0, 1, 0, 0]"
4428,"[2, 2, 5, 2, 8, 3, 7, 2, 2, 2, 8, 0, 3, 9, 5, ...","[0, 0, 1, 0]"


In [25]:
# SANITY CHECK #2

# 25% or 2000 samples should be 0,0,0,1
np.count_nonzero(df_random_with_mid_ooo_2000_2000_2000_2000_sample_number_50['outcome'].apply(lambda x: x == [0,0,0,1]))

2000

In [26]:
# SANITY CHECK #3

# The 'sample' length should be same as SINGLE_SAMPLE_SIZE_50 = BATCH_NUMBER_COUNT_50 * NUMBER_SIZE
len(df_random_with_mid_ooo_2000_2000_2000_2000_sample_number_50['sample'][1])

750

In [34]:
# Split out dataframe containing both samples and outcome

(X_2000_2000_2000_2000_with_mid_ooo_sample_number_50_train, Y_2000_2000_2000_2000_with_mid_ooo_sample_number_50_train) = prepare_train_data(df_random_with_mid_ooo_2000_2000_2000_2000_sample_number_50, SINGLE_SAMPLE_SIZE_50)

(X_10000_10000_10000_10000_with_mid_ooo_sample_number_50_train, Y_10000_10000_10000_10000_with_mid_ooo_sample_number_50_train) = prepare_train_data(df_random_with_mid_ooo_10000_10000_10000_10000_sample_number_50, SINGLE_SAMPLE_SIZE_50)

type(X):<class 'list'>, len(X):8000, len(X[0]):750
df_row_count: 8000
single_sample_size_bit:750
X_train.shape:(8000, 1, 750, 1), Y_train.shape:(8000, 4)


type(Y[0]):<class 'list'>, Y[0]:[0, 0, 1, 0]
type(X):<class 'list'>, len(X):40000, len(X[0]):750
df_row_count: 40000
single_sample_size_bit:750
X_train.shape:(40000, 1, 750, 1), Y_train.shape:(40000, 4)


type(Y[0]):<class 'list'>, Y[0]:[1, 0, 0, 0]
type(X):<class 'list'>, len(X):56000, len(X[0]):750
df_row_count: 56000
single_sample_size_bit:750
X_train.shape:(56000, 1, 750, 1), Y_train.shape:(56000, 4)


type(Y[0]):<class 'list'>, Y[0]:[0, 0, 1, 0]


In [28]:
# For CNN, the dimensions are number of samples, height, width, channel/feature maps
# Number of samples: sum of 'attack' and 'no attack' samples
# Height: 1
# Width: SINGLE_SAMPLE_SIZE_50
# Feature maps/Channels: 1
X_2000_2000_2000_2000_with_mid_ooo_sample_number_50_train.shape

(8000, 1, 750, 1)

In [29]:
# SANITY CHECK #5

# The total rows should be attack and non-attack rows
# The wide should be array of 4, since it can be [0,0,0,0], [0,0,0,1], etc.
Y_2000_2000_2000_2000_with_mid_ooo_sample_number_50_train.shape

(8000, 4)

In [10]:
# Various multi-class CNN models tested

def create_cnn_model_G_50():
  # Create model
  model = Sequential()
  # Max number of digits are different in number yet still can be within threshold
  digit_diff_size = 2
  digit_same_size = NUMBER_SIZE - digit_diff_size
  # If the seq is in the middle then parts that are same is split in two
  smallest_digit_same_size = math.floor(digit_same_size/2)
  # add model layers
  number_digit_diff_size = math.floor(NUMBER_SIZE/digit_diff_size)
  # 20190126-5000-60% model.add(Conv2D(80, kernel_size=(1,smallest_digit_same_size), strides=(1,1), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  # 20190126-5000-60% model.add(Conv2D(30, kernel_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE), activation='relu'))
  model.add(Conv2D(50, kernel_size=(1,smallest_digit_same_size), strides=(1,1), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  model.add(Conv2D(20, kernel_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE), activation='relu'))
  #model.add(Conv2D(THRESHOLD_COUNT, kernel_size=(1,NUMBER_SIZE_BIT), activation='relu'))
  model.add(Flatten())
  model.add(Dense(BATCH_NUMBER_COUNT_50, activation="relu"))
  model.add(Dense(4, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_coarse():
  # Create model
  model = Sequential()
  model.add(Conv2D(50, kernel_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  model.add(Conv2D(20, kernel_size=(1,1), strides=(1,1), activation='relu'))
  #model.add(Conv2D(THRESHOLD_COUNT, kernel_size=(1,NUMBER_SIZE_BIT), activation='relu'))
  model.add(Flatten())
  model.add(Dense(BATCH_NUMBER_COUNT_50, activation="relu"))
  model.add(Dense(4, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_coarse_2():
  # Create model
  model = Sequential()
  model.add(Conv2D(40, kernel_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1)))
  model.add(Conv2D(20, kernel_size=(1,1), strides=(1,1), activation='relu'))
  #model.add(Conv2D(THRESHOLD_COUNT, kernel_size=(1,NUMBER_SIZE_BIT), activation='relu'))
  model.add(Flatten())
  model.add(Dense(BATCH_NUMBER_COUNT_50, activation="relu"))
  model.add(Dense(4, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_coarse_3():
  # Create model
  model = Sequential()
  model.add(Conv2D(50, kernel_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1)))
  model.add(Conv2D(30, kernel_size=(1,1), strides=(1,1), activation='relu'))
  model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1)))
  #model.add(Conv2D(THRESHOLD_COUNT, kernel_size=(1,NUMBER_SIZE_BIT), activation='relu'))
  model.add(Flatten())
  model.add(Dense(BATCH_NUMBER_COUNT_50, activation="relu"))
  model.add(Dense(4, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_coarse_4():
  # Create model
  model = Sequential()
  model.add(Conv2D(50, kernel_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1)))
  model.add(Conv2D(30, kernel_size=(1,1), strides=(1,1), activation='relu'))
  model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1)))
  #model.add(Conv2D(THRESHOLD_COUNT, kernel_size=(1,NUMBER_SIZE_BIT), activation='relu'))
  model.add(Flatten())
  model.add(Dense(int(BATCH_NUMBER_COUNT_50), activation="relu"))
  model.add(Dense(4, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_mod():
  # Create model
  model = Sequential()
  # Max number of bits are different in number yet still can be within threshold
  bit_diff_size = BIT_PER_DIGIT*2
  bit_same_size = NUMBER_SIZE_BIT - bit_diff_size
  # If the seq is in the middle then parts that are same is split in two
  smallest_bit_same_size = int(math.floor(bit_same_size/2) / BIT_PER_DIGIT)
  # add model layers
  number_bit_diff_size = math.floor(NUMBER_SIZE_BIT/bit_diff_size)
  model.add(Conv2D(150, kernel_size=(1,smallest_bit_same_size), strides=(1,BIT_PER_DIGIT), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_BIT_50,1)))
  model.add(Conv2D(5, kernel_size=(1,NUMBER_SIZE_BIT ), strides=(1,NUMBER_SIZE_BIT), activation='relu'))
  model.add(Flatten())
  model.add(Dense(10 - THRESHOLD_COUNT, activation="relu"))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_maxpool():
  # Create model
  model = Sequential()
  # Max number of bits are different in number yet still can be within threshold
  digit_diff_size = 2
  digit_same_size = NUMBER_SIZE - digit_diff_size
  # If the seq is in the middle then parts that are same is split in two
  smallest_digit_same_size = math.floor(digit_same_size/2)
  # add model layers
  number_digit_diff_size = math.floor(NUMBER_SIZE/digit_diff_size)
  model.add(Conv2D(50, kernel_size=(1,smallest_digit_same_size), strides=(1,1), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  model.add(Conv2D(20, kernel_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE), activation='relu'))
  model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1)))
  model.add(Flatten())
  model.add(Dense(BATCH_NUMBER_COUNT_50, activation="relu"))
  model.add(Dense(4, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_cnn_model_G_50_avepool():
  # Create model
  model = Sequential()
  # Max number of bits are different in number yet still can be within threshold
  digit_diff_size = 2
  digit_same_size = NUMBER_SIZE - digit_diff_size
  # If the seq is in the middle then parts that are same is split in two
  smallest_digit_same_size = math.floor(digit_same_size/2)
  # add model layers
  number_digit_diff_size = math.floor(NUMBER_SIZE/digit_diff_size)
  model.add(Conv2D(50, kernel_size=(1,smallest_digit_same_size), strides=(1,1), activation='relu', input_shape=(1,SINGLE_SAMPLE_SIZE_50,1)))
  model.add(AveragePooling2D(pool_size=(1,NUMBER_SIZE), strides=(1,NUMBER_SIZE)))
  model.add(Flatten())
  model.add(Dense(BATCH_NUMBER_COUNT_50, activation="relu"))
  model.add(Dense(4, kernel_initializer='normal', activation='sigmoid'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


In [35]:
# The best is perhaps: create_cnn_model_G_50_coarse_3

cnn_model_G_50_coarse_3 = create_cnn_model_G_50_coarse_3()

cnn_model_G_50_coarse_3.fit(X_10000_10000_10000_10000_with_mid_ooo_sample_number_50_train, Y_10000_10000_10000_10000_with_mid_ooo_sample_number_50_train, validation_split=0.2, batch_size=10, epochs=50)

Train on 32000 samples, validate on 8000 samples
Epoch 1/50
32000/32000 [==============================] - 23s - loss: 1.2299 - acc: 0.3649 - val_loss: 0.9632 - val_acc: 0.5354
Epoch 2/50
32000/32000 [==============================] - 22s - loss: 0.9112 - acc: 0.5708 - val_loss: 0.8406 - val_acc: 0.6068
Epoch 3/50
32000/32000 [==============================] - 22s - loss: 0.8415 - acc: 0.6074 - val_loss: 0.8159 - val_acc: 0.6276
Epoch 4/50
32000/32000 [==============================] - 22s - loss: 0.8052 - acc: 0.6349 - val_loss: 0.7744 - val_acc: 0.6544
Epoch 5/50
32000/32000 [==============================] - 21s - loss: 0.7834 - acc: 0.6457 - val_loss: 0.7874 - val_acc: 0.6569
Epoch 6/50
32000/32000 [==============================] - 22s - loss: 0.7686 - acc: 0.6544 - val_loss: 0.7573 - val_acc: 0.6614
Epoch 7/50
32000/32000 [==============================] - 21s - loss: 0.7577 - acc: 0.6565 - val_loss: 0.7614 - val_acc: 0.6588
Epoch 8/50
32000/32000 [==============================]

In [38]:
predict_arr = cnn_model_G_50_coarse_3.predict_classes(X_2000_2000_2000_2000_with_mid_ooo_sample_number_50_train)
# Each element in 'a' will contain 1/True if prediction matches expected outcome
expected_arr = [np.where(r==1)[0][0] for r in Y_2000_2000_2000_2000_with_mid_ooo_sample_number_50_train]
a = (predict_arr==expected_arr)
# % of correct predictions
np.count_nonzero(a)/np.size(a)

7776/8000 [============================>.] - ETA: 0s

0.718125

In [37]:
# IMPORTANT: This is ONLY for saving models
model_json = cnn_model_G_50_coarse_3.to_json()
with open("cnn_model_G_50_coarse-multi_class_non_binary_sparse-tail_heavy-head_heavy_train_mid_ooo_10000-sample_71-pct-20190518.json", "w") as json_file:
    json_file.write(model_json)
cnn_model_G_50_coarse_3.save('cnn_model_G_50_coarse-multi_class_non_binary_sparse-tail_heavy-head_heavy_train_mid_ooo_10000-sample_71-pct-20190518.h5')

In [ ]:
# Additional measurements to ensure that the accuracy is not skewed by being able to
# detect attacks very well of which there are 3 classes (75%)
# but not detect non-attacks which is 1 class (25%)
# So we measure accurary with data set where 50% is non attack, and 16.67% for
# each of the 3 attack classes

In [11]:
# Load saved model
cnn_model_G_50_coarse_3 = create_cnn_model_G_50_coarse_3()
cnn_model_G_50_coarse_3.load_weights('cnn_model_G_50_coarse-multi_class_non_binary_sparse-tail_heavy-head_heavy_train_mid_ooo_10000-sample_71-pct-20190518.h5')

In [12]:
df_random_with_mid_ooo_6000_2000_2000_2000_sample_number_50 = prepare_data(['data_no_sequence_6000_sample_number_50.csv', 'data_sequence_sparse_ooo_mid_combo_2000_sample_number_50.csv', 'data_sequence_head_heavy_ooo_mid_combo_2000_sample_number_50.csv', 'data_sequence_tail_heavy_ooo_mid_combo_2000_sample_number_50.csv'], [(6000,50), (2000,50), (2000,50), (2000,50)], [CLASSES["none"], CLASSES["sparse"], CLASSES["head-heavy"], CLASSES["tail-heavy"]])

data file:data_no_sequence_6000_sample_number_50.csv, data.shape (should be (6000, 50)): (6000, 50)
data file:data_sequence_sparse_ooo_mid_combo_2000_sample_number_50.csv, data.shape (should be (2000, 50)): (2000, 50)
data file:data_sequence_head_heavy_ooo_mid_combo_2000_sample_number_50.csv, data.shape (should be (2000, 50)): (2000, 50)
data file:data_sequence_tail_heavy_ooo_mid_combo_2000_sample_number_50.csv, data.shape (should be (2000, 50)): (2000, 50)
outcome_value:[1, 0, 0, 0], outcome_values:[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
outcome_value:[0, 1, 0, 0], outcome_values:[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
outcome_value:[0, 0, 1, 0], outcome_values:[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
outcome_value:[0, 0, 0, 1], outcome_values:[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]


In [13]:
# SANITY CHECK #1

# There should be 'sample' and 'outcome' columns
# 'sample' contains all phone numbers in period broken down into individual digits
# and combined
# 'outcome' is 0 or 1 for binary
# Also there should be mixed 0 and 1 since the samples have been randomize

df_random_with_mid_ooo_6000_2000_2000_2000_sample_number_50.head(20)

,sample,outcome
7767,"[9, 3, 7, 4, 1, 9, 5, 7, 3, 7, 5, 1, 2, 4, 5, ...","[0, 1, 0, 0]"
7133,"[8, 3, 5, 1, 5, 4, 2, 4, 6, 2, 8, 9, 1, 4, 9, ...","[0, 1, 0, 0]"
11191,"[8, 4, 9, 5, 2, 9, 5, 1, 4, 7, 9, 9, 9, 0, 4, ...","[0, 0, 0, 1]"
3967,"[7, 7, 2, 1, 7, 5, 2, 5, 2, 8, 7, 6, 4, 2, 5, ...","[1, 0, 0, 0]"
969,"[6, 4, 7, 8, 4, 7, 4, 2, 2, 2, 9, 7, 7, 8, 3, ...","[1, 0, 0, 0]"
2270,"[7, 5, 7, 9, 4, 5, 9, 8, 3, 2, 1, 2, 5, 3, 1, ...","[1, 0, 0, 0]"
10134,"[1, 1, 9, 4, 6, 0, 9, 8, 9, 2, 6, 2, 9, 0, 3, ...","[0, 0, 0, 1]"
10625,"[4, 0, 6, 3, 7, 4, 5, 8, 3, 1, 1, 7, 8, 6, 9, ...","[0, 0, 0, 1]"
4479,"[4, 8, 5, 5, 9, 4, 7, 1, 1, 1, 4, 6, 2, 4, 5, ...","[1, 0, 0, 0]"
4275,"[9, 2, 9, 7, 3, 5, 0, 7, 6, 0, 8, 4, 3, 6, 1, ...","[1, 0, 0, 0]"


In [14]:
# SANITY CHECK #2a

# 50% or 6000 samples should be 1,0,0,0
np.count_nonzero(df_random_with_mid_ooo_6000_2000_2000_2000_sample_number_50['outcome'].apply(lambda x: x == [1,0,0,0]))

6000

In [15]:
# SANITY CHECK #2b

# 2000 samples should be 0,0,0,1, i.e., other classes
np.count_nonzero(df_random_with_mid_ooo_6000_2000_2000_2000_sample_number_50['outcome'].apply(lambda x: x == [0,0,0,1]))

2000

In [16]:
# SANITY CHECK #3

# The 'sample' length should be same as SINGLE_SAMPLE_SIZE_50 = BATCH_NUMBER_COUNT_50 * NUMBER_SIZE
len(df_random_with_mid_ooo_6000_2000_2000_2000_sample_number_50['sample'][1])

750

In [17]:
(X_6000_2000_2000_2000_with_mid_ooo_sample_number_50_train, Y_6000_2000_2000_2000_with_mid_ooo_sample_number_50_train) = prepare_train_data(df_random_with_mid_ooo_6000_2000_2000_2000_sample_number_50, SINGLE_SAMPLE_SIZE_50)

type(X):<class 'list'>, len(X):12000, len(X[0]):750
df_row_count: 12000
single_sample_size_bit:750
X_train.shape:(12000, 1, 750, 1), Y_train.shape:(12000, 4)


type(Y[0]):<class 'list'>, Y[0]:[0, 1, 0, 0]


In [18]:
# For CNN, the dimensions are number of samples, height, width, channel/feature maps
# Number of samples: sum of 'attack' and 'no attack' samples
# Height: 1
# Width: SINGLE_SAMPLE_SIZE_50
# Feature maps/Channels: 1
X_6000_2000_2000_2000_with_mid_ooo_sample_number_50_train.shape

(12000, 1, 750, 1)

In [19]:
# SANITY CHECK #5

# The total rows should be attack and non-attack rows
# The wide should be array of 4, since it can be [0,0,0,0], [0,0,0,1], etc.
Y_6000_2000_2000_2000_with_mid_ooo_sample_number_50_train.shape

(12000, 4)

In [20]:
predict_arr = cnn_model_G_50_coarse_3.predict_classes(X_6000_2000_2000_2000_with_mid_ooo_sample_number_50_train)
# Each element in 'a' will contain 1/True if prediction matches expected outcome
expected_arr = [np.where(r==1)[0][0] for r in Y_6000_2000_2000_2000_with_mid_ooo_sample_number_50_train]
a = (predict_arr==expected_arr)
# % of correct predictions
np.count_nonzero(a)/np.size(a)

11872/12000 [============================>.] - ETA: 0s

0.7189166666666666